# PySpark para Filtragem Colaborativa
## ALS

In [0]:
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

In [0]:
from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [0]:
ratings = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .parquet("/FileStore/tables/recomendacao/ratings.parquet")

In [0]:
ratings.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
|     1|   1590|   2.5|1256677236|
|     1|   1591|   1.5|1256677475|
|     1|   2134|   4.5|1256677464|
|     1|   2478|   4.0|1256677239|
|     1|   2840|   3.0|1256677500|
+------+-------+------+----------+
only showing top 10 rows



In [0]:
movies = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .csv("/FileStore/tables/recomendacao/movies.csv")

In [0]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [0]:
ratings.printSchema()

root
 |-- userId: long (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)



In [0]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')

## Calculando Esparsidade

In [0]:
# Conta o número total de avaliações no dataset
numerador = ratings.select("rating").count()

# Conta o número de usuários e filmes distintos 
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# O denominador vai ser o produto do número de filmes pelo número de usuários únicos
denominador = num_users * num_movies

# Divide o numerador pelo denominador
sparsity = (1.0 - (numerador *1.0)/denominador)*100
print("O dataframe de filmes é ", "%.2f" % sparsity + "% empty.")

O dataframe de filmes é  99.82% empty.


## Interpretando as Ratings

In [0]:
# Agrupando o número de vezes cada usuários avaliou um filme
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|123100|23715|
|117490| 9279|
|134596| 8381|
|212343| 7884|
|242683| 7515|
|111908| 6645|
| 77609| 6398|
| 63783| 6346|
|172357| 5868|
|141955| 5810|
|158002| 5701|
|253511| 5356|
| 48470| 5257|
|183233| 5169|
| 94843| 5130|
| 73145| 5042|
| 37046| 5041|
|187986| 4951|
|  4796| 4874|
|236981| 4854|
+------+-----+
only showing top 20 rows



In [0]:
subset_userId_ratings = userId_ratings.filter("count > 500")

In [0]:
lista_subset_id = subset_userId_ratings.select('userId').rdd.flatMap(lambda x: x).collect()

In [0]:
ratings = ratings.filter(col('userId').isin(lista_subset_id))

In [0]:
# Agrupando para ver o número de vezes cada filme foi avaliado
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|   2571| 9542|
|    356| 9411|
|    296| 9198|
|    480| 9103|
|   1270| 9102|
|    260| 9069|
|    593| 9020|
|   1580| 8963|
|   1196| 8905|
|    318| 8855|
|   1198| 8851|
|   2762| 8785|
|   1210| 8668|
|      1| 8569|
|   2858| 8546|
|    589| 8522|
|   1265| 8480|
|   2959| 8465|
|   4993| 8452|
|   3578| 8404|
+-------+-----+
only showing top 20 rows



## Gerando Customer ID

O ALS **precisa** que todas as entradas sejam números, dessa forma, se você estiver lidando com algum id criptografado ou até mesmo seu id seja uma string (em vez do id do filmes fossemos lidar com o nome do filme). Você tera que criar algum tipo de label encoder para sua variável de id ser agora um inteiro. O PySpark tem uma versão do label encoder, porém ela é muito custosa, estourando a memoria com facilidade para bases muito grandes. Dessa forma, iremos usar uma alternativa menos custosa.

In [0]:
# Usando label encoder
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="movie_name", outputCol="movie_name_id") 
indexed = indexer.fit(df).transform(df) 

In [0]:
from pyspark.sql.functions import  monotonically_increasing_id

In [0]:
customer_cpf = ratings.select('userId').distinct().coalesce(1)

In [0]:
customer_cpf.show(5)

+------+
|userId|
+------+
|   148|
|   463|
|   471|
|   496|
|   833|
+------+
only showing top 5 rows



In [0]:
customer_cpf = customer_cpf.withColumn('customer_id', monotonically_increasing_id()).persist()

In [0]:
data = df.join(customer_cpf, how='inner', on='cpf')

#Construindo ALS

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", rank = 10, maxIter = 5, regParam = .05, alpha = 40, nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

Out[24]: pyspark.ml.recommendation.ALS

- Rank: Número de Features Latentes da nossa matriz
- maxIter: Número de iterações (Parecido com epochs)
- regParam: Lambda
- alpha: Discutiremos depois (Pois é um hiperparâmetro para variáveis implicitas)
- nonnegative = True : Apenas toma números positivos
- condStartStrategy = "drop": Pode acabar tendo usuários com dados no treino de teste
- implicitPrefs = True: Diz ao modelo se estamos trabalhando com dados implicitos ou explicitos.

In [0]:
#Fit modelo ALS
model = als.fit(train)

In [0]:
#Gerando previsões no conjunto de teste
predictions = model.transform(test)

#Tunando o ALS

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# adiciona hiperparâmetros e seus respectivos valores
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Número de modelos a ser testado: ", len(param_grid))

Número de modelos a ser testado:  16


#Construindo uma pipeline com cross validation

In [0]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_e154a763d34e


## Melhor modelo e melhor parâmetro

In [0]:
#Fit cross validator ao conjunto de treino
model = cv.fit(train)

#Extrai o melhor modelo do cv acima
best_model = model.bestModel

In [0]:
# Print best_model
print(type(best_model))

# O código abaixo vai retornar os valores que realizaram o menor RMSE
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

In [0]:
#Caso tenha usado o best model, para testar é dessa forma:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

In [0]:
predictions.show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|  6336|     10|   2.0| 3.1934292|
|  6336|     16|   3.5| 3.8623238|
|  6336|     32|   3.5| 3.9014945|
|  6336|     34|   4.0|  3.661192|
|  6336|    103|   3.0| 2.7465103|
|  6336|    111|   4.0| 4.0222754|
|  6336|    121|   4.0| 3.6936579|
|  6336|    170|   1.0|  2.817999|
|  6336|    173|   3.0| 1.9595488|
|  6336|    296|   5.0|  4.392178|
+------+-------+------+----------+
only showing top 10 rows



In [0]:
RMSE = evaluator.evaluate(predictions)
print(RMSE)

0.7727216267619591


#Fazendo Recomendações

In [0]:
# Gera n Recomendações para todos os usuários
nrecommendations = model.recommendForAllUsers(3)

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
nrecommendations.limit(5).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    81|[{145893, 6.66362...|
|   332|[{141532, 5.91178...|
|   458|[{176261, 4.61375...|
|   540|[{182287, 6.09249...|
|   593|[{153184, 5.71026...|
+------+--------------------+



In [0]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

In [0]:
nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|    81| 145893| 6.663622|
|    81| 153184| 6.499804|
|    81|  80719| 6.375702|
|   332| 141532| 5.911784|
|   332| 133881| 5.888237|
|   332| 103112|5.8649526|
|   458| 176261|4.6137514|
|   458| 128618|4.5275064|
|   458|  87719| 4.496281|
|   540| 182287|  6.09249|
+------+-------+---------+



In [0]:
#Fazendo recomendações para os itens
movieRecs = model.recommendForAllItems(10)

In [0]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('movieId', col("rec_exp.userId"), col("rec_exp.rating"))

In [0]:
# Gera top 10 recomendações para um conjunto de usuários
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

In [0]:
# Gera top 10 recomendações para um conjunto de filmes 
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)